#  환경

In [3]:
import os, glob, time
import pandas as pd

# 크롤링
import requests
import urllib.request

from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.parse import quote

In [4]:
# 저장된 종목 번호 가지고 오기 
import pickle

# load data
with open('stock_num_list.pickle', 'rb') as fr:
    stockInfo = pickle.load(fr)

In [25]:
# 1. 종목 이름과 번호를 매치 시킨다.  
def search_data(inPut):
    """
    1. 주식 조회를 가능하게 한다. 
    inPut에 종목이름을 작성하면 종목번호를 리턴한다. 
    """
#     inPut = input("주식명")
    stockNumber = stockInfo[inPut]
    return stockNumber

######################################################################

# 2. 해당 url 의 정보를 가지고 온다 
def PageUrl( stockNumber, choiceNum, page ):
    """
    # Page assignment
    choiceNum -> 1 -> 메인 , 2 -> 게시판
    stockNumber -> 종목번호
    page = 페이지번호
    """

    def choice(choiceNum):
        """
        1 -> 메인 , 2 -> 게시판
        """
        choiceNum = str(choiceNum)

        if choiceNum == "1":
            subject = "main"
            return(subject)
        elif choiceNum == "2":
            subject = "board"
            return(subject)
        
    mainUrl    = f'https://finance.naver.com'
    subject    = choice(choiceNum)
    
    if page == 1 :
        #  page data
        paramUrl  = f"/item/{subject}.nhn?code={stockNumber}"
        targetUrl = mainUrl + paramUrl
        return targetUrl

    else:
        # Non-page data
        paramUrl  = f"/item/{subject}.nhn?code={stockNumber}&page=" + str(page)
        targetUrl = mainUrl + paramUrl
        return targetUrl
    
######################################################################

# 3. 리퀘스트 문을 작성한다. 
def req_point(searchUrl):
    """
    리쿼스트 전용 
    """
    try:
        req  = requests.get( searchUrl , headers={'User-Agent':'Mozilla/5.0'} )
    except:
        req  = requests.get( searchUrl )

    html    = req.text
    soup    = BeautifulSoup( html, 'html.parser' )
    
    return soup


######################################################################


# 4. 보드부터 진행한다. 
# 보드에서 게시물 정보를 수집한다. 
# 일차적으로 게시물 이름과 각 게시물의 url을 가지고 온다. 

def board_crw(mainUrl):
    Today = time.strftime('%Y.%m.%d', time.localtime(time.time()))
#     Today_dev = '2020.09.01'
    """
    mainUrl의 인자를 받아서 
    보드에서 게시물 정보를 수집한다. 
    몇부분 수집하기 힘든 부분은 .. 
    """
    chBox = [] 
    urlBox = []   
    
    soup   = req_point(mainUrl)

    for j in soup.find("tbody").find_all('tr'):
        for sp in j.find_all('span',{'class':'tah p10 gray03'}):
#             print(sp.text)
            if sp.text.split(' ')[0]== Today :
                print('오늘')
                if sp.text[:4] == '2020' :
                    chBox.append(sp.text)
                    print(len(chBox))
            else:
                print('empty')
                break
       
    for i in soup.find("tbody").find_all('a'):
        if len(chBox) != len(urlBox):  
            urlBox.append(i['href'])
        else:
            break
                
    return urlBox, chBox


######################################################################


# 5. 각 게시물 안에서 추가적으로 게시물 정보를 수집한다. 
# 시간이 조금걸리는데 조금도 빠르게 수집하는 방법을 알아봐야 할것같다. 

def BoardCont( bdUrl ):
    """
    작은 보드 - 제목 ,날짜,본문
    """    
    titBox = list()
    txtBox = list()
    datBox = list()
    
    try:
    
        mainUrl    = f'https://finance.naver.com'
        smallContentsUrl = mainUrl + bdUrl
        print('URL : ',smallContentsUrl)
        soup      = req_point(smallContentsUrl)

        # 타이틀 
        for title in soup.find_all("strong", {"class":"c p15"}):
            tit = title.text
            print("tit : ",tit)
            titBox.append(tit)

        # 내용 
        for cont in soup.find("div", {"id":"body"}):
            txt = re.sub('<.+ ?>', '', str(cont)).strip()
            txtBox.append(txt)

        # 날짜 -> 만약에 오늘 자 아님 멈추는 코드 필요 
        for date in soup.find_all("th", {"class":"gray03 p9 tah"}):
            print('date: ',date.text)
            datBox.append(date.text)   

        return titBox , datBox, txtBox
    
    except Exception as err:
        print('<< Error : URL을 확인해주세요. >>','\n', err)

######################################################################



In [19]:
#  choiceNum = "게시판" , stockNumber = "주식종목번호" , 페이지 
Name = "와이팜"
choiceNum = 2
stockNumber = search_data(Name)
page = 1
mainUrl =  PageUrl( stockNumber, choiceNum, page )
print(mainUrl)

https://finance.naver.com/item/board.nhn?code=332570


In [20]:
urlBox, chBox  = board_crw(mainUrl)

empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty


In [21]:
type(urlBox)

list

In [26]:
titBox , datBox, txtBox = BoardCont(urlBox)

<< Error : URL을 확인해주세요. >> 
 can only concatenate str (not "list") to str


TypeError: cannot unpack non-iterable NoneType object

In [21]:
# 데이터 프레임으로 만들기 
urlBox = board_crw(mainUrl) # 게시물 경로 가지고오기 
titBox , datBox, txtBox = BoardCont(urlBox[0]) # 게시물 가지고오기 

In [ ]:
# 데이터 프레임에 담고 -> 도는동안 다른 크롤링 도전해보기 

In [224]:
# # 데이터 프레임 

# df = pd.DataFrame({
# #     'name' : []
# })